# Tuning Neural Networks with Normalization - Lab 

## Introduction

In this lab you'll build a neural network to perform a regression task.

It is worth noting that getting regression to work with neural networks can be comparatively difficult because the output is unbounded ($\hat y$ can technically range from $-\infty$ to $+\infty$), and the models are especially prone to exploding gradients. This issue makes a regression exercise the perfect learning case for tinkering with normalization and optimization strategies to ensure proper convergence!

## Objectives

In this lab you will: 

- Fit a neural network to normalized data 
- Implement and observe the impact of various initialization techniques 
- Implement and observe the impact of various optimization techniques 

## Load the data 

First, run the following cell to import all the neccessary libraries and classes you will need in this lab. 

In [1]:
# Necessary libraries and classes
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras import initializers
from keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from keras import optimizers
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In this lab, you'll be working with the housing prices data you saw in an earlier section. However, we did a lot of preprocessing for you so you can focus on normalizing numeric features and building neural network models! The following preprocessing steps were taken (all the code can be found in the `data_preprocessing.ipynb` notebook in this repository): 

- The data was split into the training, validate, and test sets 
- All the missing values in numeric columns were replaced by the median of those columns 
- All the missing values in catetgorical columns were replaced with the word 'missing' 
- All the categorical columns were one-hot encoded 

Run the following cells to import the train, validate, and test sets:  

In [2]:
# Load all numeric features
X_train_numeric = pd.read_csv('data/X_train_numeric.csv')
X_val_numeric = pd.read_csv('data/X_val_numeric.csv')
X_test_numeric = pd.read_csv('data/X_test_numeric.csv')

# Load all categorical features
X_train_cat = pd.read_csv('data/X_train_cat.csv')
X_val_cat = pd.read_csv('data/X_val_cat.csv')
X_test_cat = pd.read_csv('data/X_test_cat.csv')

# Load all targets
y_train = pd.read_csv('data/y_train.csv')
y_val = pd.read_csv('data/y_val.csv')
y_test = pd.read_csv('data/y_test.csv')

In [3]:
# Combine all features
X_train = pd.concat([X_train_numeric, X_train_cat], axis=1)
X_val = pd.concat([X_val_numeric, X_val_cat], axis=1)
X_test = pd.concat([X_test_numeric, X_test_cat], axis=1)

# Number of features
n_features = X_train.shape[1]

As a refresher, preview the training data: 

In [4]:
# Preview the data
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,80.0,69.0,21453.0,6.0,5.0,1969.0,1969.0,0.0,938.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,60.0,79.0,12420.0,7.0,5.0,2001.0,2001.0,0.0,666.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,20.0,75.0,9742.0,8.0,5.0,2002.0,2002.0,281.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,120.0,39.0,5389.0,8.0,5.0,1995.0,1996.0,0.0,1180.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,60.0,85.0,11003.0,10.0,5.0,2008.0,2008.0,160.0,765.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


## Build a Baseline Model

Building a naive baseline model to compare performance against is a helpful reference point. From there, you can then observe the impact of various tunning procedures which will iteratively improve your model. So, let's do just that! 

In the cell below: 

- Add an input layer with `n_features` units 
- Add two hidden layers, one with 100 and the other with 50 units (make sure you use the `'relu'` activation function) 
- Add an output layer with 1 unit and `'linear'` activation 
- Compile and fit the model 

In [7]:
np.random.seed(123)
baseline_model = Sequential()

# Hidden layer with 100 units
baseline_model.add(layers.Dense(100, activation='relu', input_shape = (n_features,)))

# Hidden layer with 50 units
baseline_model.add(layers.Dense(50, activation='relu'))

# Output layer
baseline_model.add(layers.Dense(1, activation='linear'))

# Compile the model
baseline_model.compile(optimizer='SGD', 
                       loss='mse', 
                       metrics=['mse'])

# Train the model
baseline_model.fit(X_train, 
                   y_train, 
                   batch_size=32, 
                   epochs=150, 
                   validation_data=(X_val, y_val))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 1s 655us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
1051/1051 [==============================] - 0s 159us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
1051/1051 [==============================] - 0s 173us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
1051/1051 [==============================] - 0s 162us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
1051/1051 [==============================] - 0s 183us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
1051/1051 [==============================] - 0s 168us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
1051/1051 [==============================] - 0s 176us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
1051/1051 [==============================] - 0s 183us/step - loss:

Epoch 66/150
1051/1051 [==============================] - 0s 162us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 67/150
1051/1051 [==============================] - 0s 138us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 68/150
1051/1051 [==============================] - 0s 140us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 69/150
1051/1051 [==============================] - 0s 139us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 70/150
1051/1051 [==============================] - 0s 137us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
1051/1051 [==============================] - 0s 136us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
1051/1051 [==============================] - 0s 134us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 73/150
1051/1051 [==============================] - 0s 146us/step - loss: nan - mse: nan - val_loss: nan - val_m

Epoch 131/150
1051/1051 [==============================] - 0s 156us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 132/150
1051/1051 [==============================] - 0s 153us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 133/150
1051/1051 [==============================] - 0s 150us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 134/150
1051/1051 [==============================] - 0s 155us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 135/150
1051/1051 [==============================] - 0s 155us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 136/150
1051/1051 [==============================] - 0s 153us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 137/150
1051/1051 [==============================] - 0s 151us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 138/150
1051/1051 [==============================] - 0s 152us/step - loss: nan - mse: nan - val_loss: nan

> _**Notice this extremely problematic behavior: all the values for training and validation loss are "nan". This indicates that the algorithm did not converge. The first solution to this is to normalize the input. From there, if convergence is not achieved, normalizing the output may also be required.**_ 

## Normalize the Input Data 

It's now time to normalize the input data. In the cell below: 

- Assign the column names of all numeric columns to `numeric_columns` 
- Instantiate a `StandardScaler` 
- Fit and transform `X_train_numeric`. Make sure you convert the result into a DataFrame (use `numeric_columns` as the column names) 
- Transform validate and test sets (`X_val_numeric` and `X_test_numeric`), and convert these results into DataFrames as well 
- Use the provided to combine the scaled numerical and categorical features 

In [8]:
# Numeric column names
numeric_columns = X_train_numeric.columns 

# Instantiate StandardScaler
ss_X = StandardScaler()

# Fit and transform train data
X_train_scaled = pd.DataFrame(ss_X.fit_transform(X_train_numeric), columns=numeric_columns)

# Transform validate and test data
X_val_scaled = pd.DataFrame(ss_X.fit_transform(X_val_numeric), columns=numeric_columns)
X_test_scaled =pd.DataFrame(ss_X.fit_transform(X_test_numeric), columns=numeric_columns)

# Combine the scaled numerical features and categorical features
X_train = pd.concat([X_train_scaled, X_train_cat], axis=1)
X_val = pd.concat([X_val_scaled, X_val_cat], axis=1)
X_test = pd.concat([X_test_scaled, X_test_cat], axis=1)

Now run the following cell to compile a neural network model (with the same architecture as before): 

In [9]:
# Model with all normalized inputs
np.random.seed(123)
normalized_input_model = Sequential()
normalized_input_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
normalized_input_model.add(layers.Dense(50, activation='relu'))
normalized_input_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_input_model.compile(optimizer='SGD', 
                               loss='mse', 
                               metrics=['mse'])

In the cell below: 
- Train the `normalized_input_model` on normalized input (`X_train`) and output (`y_train`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` argument as `(X_val, y_val)` 

In [10]:
# Train the model 
normalized_input_model.fit(X_train, y_train, epochs=150, batch_size=32, validation_data=(X_val, y_val))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 354us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
1051/1051 [==============================] - 0s 151us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
1051/1051 [==============================] - 0s 125us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
1051/1051 [==============================] - 0s 133us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
1051/1051 [==============================] - 0s 125us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
1051/1051 [==============================] - 0s 125us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
1051/1051 [==============================] - 0s 128us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
1051/1051 [==============================] - 0s 126us/step - loss:

Epoch 66/150
1051/1051 [==============================] - 0s 137us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 67/150
1051/1051 [==============================] - 0s 138us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 68/150
1051/1051 [==============================] - 0s 132us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 69/150
1051/1051 [==============================] - 0s 132us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 70/150
1051/1051 [==============================] - 0s 145us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
1051/1051 [==============================] - 0s 151us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
1051/1051 [==============================] - 0s 157us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 73/150
1051/1051 [==============================] - 0s 152us/step - loss: nan - mse: nan - val_loss: nan - val_m

Epoch 131/150
1051/1051 [==============================] - 0s 199us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 132/150
1051/1051 [==============================] - 0s 184us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 133/150
1051/1051 [==============================] - 0s 158us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 134/150
1051/1051 [==============================] - 0s 180us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 135/150
1051/1051 [==============================] - 0s 224us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 136/150
1051/1051 [==============================] - 0s 170us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 137/150
1051/1051 [==============================] - 0s 159us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 138/150
1051/1051 [==============================] - 0s 233us/step - loss: nan - mse: nan - val_loss: nan

> _**Note that you still haven't achieved convergence! From here, it's time to normalize the output data.**_

## Normalizing the output

Again, use `StandardScaler()` to: 

- Fit and transform `y_train` 
- Transform `y_val` and `y_test` 

In [11]:
# Instantiate StandardScaler
ss_y = StandardScaler()

# Fit and transform train labels
y_train_scaled = ss_y.fit_transform(y_train)

# Transform validate and test labels
y_val_scaled = ss_y.fit_transform(y_val)
y_test_scaled = ss_y.fit_transform(y_test)

In the cell below: 
- Train the `normalized_model` on normalized input (`X_train`) and output (`y_train_scaled`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` as `(X_val, y_val_scaled)` 

In [14]:
# Model with all normalized inputs and outputs
np.random.seed(123)
normalized_model = Sequential()
normalized_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
normalized_model.add(layers.Dense(50, activation='relu'))
normalized_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_model.compile(optimizer='SGD', 
                         loss='mse', 
                         metrics=['mse']) 

# Train the model
normalized_model.fit(X_train, 
                     y_train_scaled, 
                     batch_size=32, 
                     epochs=150, 
                     validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 337us/step - loss: 0.5035 - mse: 0.5035 - val_loss: 0.2389 - val_mse: 0.2389
Epoch 2/150
1051/1051 [==============================] - 0s 127us/step - loss: 0.2178 - mse: 0.2178 - val_loss: 0.1992 - val_mse: 0.1992
Epoch 3/150
1051/1051 [==============================] - 0s 112us/step - loss: 0.1803 - mse: 0.1803 - val_loss: 0.1904 - val_mse: 0.1904
Epoch 4/150
1051/1051 [==============================] - 0s 131us/step - loss: 0.1618 - mse: 0.1618 - val_loss: 0.1617 - val_mse: 0.1617
Epoch 5/150
1051/1051 [==============================] - 0s 123us/step - loss: 0.1464 - mse: 0.1464 - val_loss: 0.1540 - val_mse: 0.1540
Epoch 6/150
1051/1051 [==============================] - 0s 118us/step - loss: 0.1325 - mse: 0.1325 - val_loss: 0.1453 - val_mse: 0.1453
Epoch 7/150
1051/1051 [==============================] - 0s 124us/step - loss: 0.1260 - mse: 0.1260 - val_loss: 0.1508 - val_mse: 0

1051/1051 [==============================] - 0s 153us/step - loss: 0.0273 - mse: 0.0273 - val_loss: 0.1266 - val_mse: 0.1266
Epoch 61/150
1051/1051 [==============================] - 0s 139us/step - loss: 0.0276 - mse: 0.0276 - val_loss: 0.1252 - val_mse: 0.1252
Epoch 62/150
1051/1051 [==============================] - 0s 114us/step - loss: 0.0264 - mse: 0.0264 - val_loss: 0.1268 - val_mse: 0.1268
Epoch 63/150
1051/1051 [==============================] - 0s 131us/step - loss: 0.0271 - mse: 0.0271 - val_loss: 0.1354 - val_mse: 0.1354
Epoch 64/150
1051/1051 [==============================] - 0s 111us/step - loss: 0.0268 - mse: 0.0268 - val_loss: 0.1267 - val_mse: 0.1267
Epoch 65/150
1051/1051 [==============================] - 0s 111us/step - loss: 0.0256 - mse: 0.0256 - val_loss: 0.1279 - val_mse: 0.1279
Epoch 66/150
1051/1051 [==============================] - 0s 137us/step - loss: 0.0249 - mse: 0.0249 - val_loss: 0.1267 - val_mse: 0.1267
Epoch 67/150
1051/1051 [=======================

1051/1051 [==============================] - 0s 177us/step - loss: 0.0139 - mse: 0.0139 - val_loss: 0.1270 - val_mse: 0.1270
Epoch 120/150
1051/1051 [==============================] - 0s 159us/step - loss: 0.0139 - mse: 0.0139 - val_loss: 0.1290 - val_mse: 0.1290
Epoch 121/150
1051/1051 [==============================] - 0s 159us/step - loss: 0.0138 - mse: 0.0138 - val_loss: 0.1260 - val_mse: 0.1260
Epoch 122/150
1051/1051 [==============================] - 0s 162us/step - loss: 0.0140 - mse: 0.0140 - val_loss: 0.1283 - val_mse: 0.1283
Epoch 123/150
1051/1051 [==============================] - 0s 167us/step - loss: 0.0135 - mse: 0.0135 - val_loss: 0.1289 - val_mse: 0.1289
Epoch 124/150
1051/1051 [==============================] - 0s 161us/step - loss: 0.0137 - mse: 0.0137 - val_loss: 0.1272 - val_mse: 0.1272
Epoch 125/150
1051/1051 [==============================] - 0s 169us/step - loss: 0.0135 - mse: 0.0135 - val_loss: 0.1275 - val_mse: 0.1275
Epoch 126/150
1051/1051 [================

Nicely done! After normalizing both the input and output, the model finally converged. 

- Evaluate the model (`normalized_model`) on training data (`X_train` and `y_train_scaled`) 

In [15]:
# Evaluate the model on training data
normalized_model.evaluate(X_train, y_train_scaled)

1051/1051 [==============================] - 0s 108us/step


[0.010184351599409284, 0.010184352286159992]

- Evaluate the model (`normalized_model`) on validate data (`X_val` and `y_val_scaled`) 

In [16]:
# Evaluate the model on validate data
normalized_model.evaluate(X_val, y_val_scaled)

263/263 [==============================] - 0s 87us/step


[0.12862504391219226, 0.1286250501871109]

Since the output is normalized, the metric above is not interpretable. To remedy this: 

- Generate predictions on validate data (`X_val`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_val` and `y_val_pred` 

In [17]:
# Generate predictions on validate data
y_val_pred_scaled = normalized_model.predict(X_val)

# Transform the predictions back to original scale
y_val_pred = ss_y.inverse_transform(y_val_pred_scaled)

# RMSE of validate data
np.sqrt(mean_squared_error(y_val, y_val_pred))

33770.45299570231

Great! Now that you have a converged model, you can also experiment with alternative optimizers and initialization strategies to see if you can find a better global minimum. (After all, the current models may have converged to a local minimum.) 

## Using Weight Initializers

In this section you will to use alternative initialization and optimization strategies. At the end, you'll then be asked to select the model which you believe performs the best.  

##  He Initialization

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='he_normal'`  

In [19]:
np.random.seed(123)
he_model = Sequential()

# Add the first hidden layer
he_model.add(layers.Dense(100, activation = 'relu', input_shape = (n_features,), kernel_initializer = 'he_normal'))

# Add another hidden layer
he_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
he_model.add(layers.Dense(1, activation='linear'))

# Compile the model
he_model.compile(optimizer='SGD', 
                 loss='mse', 
                 metrics=['mse'])

# Train the model
he_model.fit(X_train, 
             y_train_scaled, 
             batch_size=32, 
             epochs=150, 
             validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 373us/step - loss: 0.5880 - mse: 0.5880 - val_loss: 0.2951 - val_mse: 0.2951
Epoch 2/150
1051/1051 [==============================] - 0s 140us/step - loss: 0.2799 - mse: 0.2799 - val_loss: 0.2323 - val_mse: 0.2323
Epoch 3/150
1051/1051 [==============================] - 0s 140us/step - loss: 0.2158 - mse: 0.2158 - val_loss: 0.2343 - val_mse: 0.2343
Epoch 4/150
1051/1051 [==============================] - 0s 119us/step - loss: 0.1875 - mse: 0.1875 - val_loss: 0.1822 - val_mse: 0.1822
Epoch 5/150
1051/1051 [==============================] - 0s 128us/step - loss: 0.1613 - mse: 0.1613 - val_loss: 0.1766 - val_mse: 0.1766
Epoch 6/150
1051/1051 [==============================] - 0s 126us/step - loss: 0.1408 - mse: 0.1408 - val_loss: 0.1636 - val_mse: 0.1636
Epoch 7/150
1051/1051 [==============================] - 0s 138us/step - loss: 0.1345 - mse: 0.1345 - val_loss: 0.1781 - val_mse: 0

1051/1051 [==============================] - 0s 138us/step - loss: 0.0269 - mse: 0.0269 - val_loss: 0.1504 - val_mse: 0.1504
Epoch 61/150
1051/1051 [==============================] - 0s 136us/step - loss: 0.0264 - mse: 0.0264 - val_loss: 0.1508 - val_mse: 0.1508
Epoch 62/150
1051/1051 [==============================] - 0s 136us/step - loss: 0.0263 - mse: 0.0263 - val_loss: 0.1521 - val_mse: 0.1521
Epoch 63/150
1051/1051 [==============================] - 0s 136us/step - loss: 0.0260 - mse: 0.0260 - val_loss: 0.1523 - val_mse: 0.1523
Epoch 64/150
1051/1051 [==============================] - 0s 133us/step - loss: 0.0256 - mse: 0.0256 - val_loss: 0.1510 - val_mse: 0.1510
Epoch 65/150
1051/1051 [==============================] - 0s 212us/step - loss: 0.0252 - mse: 0.0252 - val_loss: 0.1503 - val_mse: 0.1503
Epoch 66/150
1051/1051 [==============================] - 0s 138us/step - loss: 0.0244 - mse: 0.0244 - val_loss: 0.1510 - val_mse: 0.1510
Epoch 67/150
1051/1051 [=======================

1051/1051 [==============================] - 0s 159us/step - loss: 0.0128 - mse: 0.0128 - val_loss: 0.1497 - val_mse: 0.1497
Epoch 120/150
1051/1051 [==============================] - 0s 157us/step - loss: 0.0126 - mse: 0.0126 - val_loss: 0.1501 - val_mse: 0.1501
Epoch 121/150
1051/1051 [==============================] - 0s 157us/step - loss: 0.0127 - mse: 0.0127 - val_loss: 0.1493 - val_mse: 0.1493
Epoch 122/150
1051/1051 [==============================] - 0s 164us/step - loss: 0.0125 - mse: 0.0125 - val_loss: 0.1494 - val_mse: 0.1494
Epoch 123/150
1051/1051 [==============================] - 0s 159us/step - loss: 0.0124 - mse: 0.0124 - val_loss: 0.1492 - val_mse: 0.1492
Epoch 124/150
1051/1051 [==============================] - 0s 157us/step - loss: 0.0121 - mse: 0.0121 - val_loss: 0.1496 - val_mse: 0.1496
Epoch 125/150
1051/1051 [==============================] - 0s 169us/step - loss: 0.0121 - mse: 0.0121 - val_loss: 0.1503 - val_mse: 0.1503
Epoch 126/150
1051/1051 [================

Evaluate the model (`he_model`) on training data (`X_train` and `y_train_scaled`) 

In [20]:
# Evaluate the model on training data
he_model.evaluate(X_train, y_train_scaled)

1051/1051 [==============================] - 0s 117us/step


[0.008822411237608865, 0.008822411298751831]

Evaluate the model (`he_model`) on validate data (`X_train` and `y_train_scaled`) 

In [21]:
# Evaluate the model on validate data
he_model.evaluate(X_val, y_val_scaled)

263/263 [==============================] - 0s 114us/step


[0.1501240405349677, 0.15012405812740326]

## Lecun Initialization 

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='lecun_normal'` 

In [23]:
np.random.seed(123)
lecun_model = Sequential()

# Add the first hidden layer
lecun_model.add(layers.Dense(100, activation='relu', input_shape = (n_features, ), kernel_initializer='lecun_normal'))

# Add another hidden layer
lecun_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
lecun_model.add(layers.Dense(1, activation='linear'))

# Compile the model
lecun_model.compile(optimizer='SGD', 
                    loss='mse', 
                    metrics=['mse'])

# Train the model
lecun_model.fit(X_train, 
                y_train_scaled, 
                batch_size=32, 
                epochs=150, 
                validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 433us/step - loss: 0.5336 - mse: 0.5336 - val_loss: 0.2258 - val_mse: 0.2258
Epoch 2/150
1051/1051 [==============================] - 0s 157us/step - loss: 0.2333 - mse: 0.2333 - val_loss: 0.1783 - val_mse: 0.1783
Epoch 3/150
1051/1051 [==============================] - 0s 164us/step - loss: 0.1834 - mse: 0.1834 - val_loss: 0.1822 - val_mse: 0.1822
Epoch 4/150
1051/1051 [==============================] - 0s 186us/step - loss: 0.1572 - mse: 0.1572 - val_loss: 0.1368 - val_mse: 0.1368
Epoch 5/150
1051/1051 [==============================] - 0s 164us/step - loss: 0.1378 - mse: 0.1378 - val_loss: 0.1331 - val_mse: 0.1331
Epoch 6/150
1051/1051 [==============================] - 0s 159us/step - loss: 0.1208 - mse: 0.1208 - val_loss: 0.1266 - val_mse: 0.1266
Epoch 7/150
1051/1051 [==============================] - 0s 162us/step - loss: 0.1137 - mse: 0.1137 - val_loss: 0.1362 - val_mse: 0

1051/1051 [==============================] - 0s 138us/step - loss: 0.0237 - mse: 0.0237 - val_loss: 0.1157 - val_mse: 0.1157
Epoch 61/150
1051/1051 [==============================] - 0s 140us/step - loss: 0.0236 - mse: 0.0236 - val_loss: 0.1129 - val_mse: 0.1129
Epoch 62/150
1051/1051 [==============================] - 0s 155us/step - loss: 0.0234 - mse: 0.0234 - val_loss: 0.1128 - val_mse: 0.1128
Epoch 63/150
1051/1051 [==============================] - 0s 150us/step - loss: 0.0233 - mse: 0.0233 - val_loss: 0.1180 - val_mse: 0.1180
Epoch 64/150
1051/1051 [==============================] - 0s 152us/step - loss: 0.0228 - mse: 0.0228 - val_loss: 0.1158 - val_mse: 0.1158
Epoch 65/150
1051/1051 [==============================] - 0s 147us/step - loss: 0.0224 - mse: 0.0224 - val_loss: 0.1146 - val_mse: 0.1146
Epoch 66/150
1051/1051 [==============================] - 0s 147us/step - loss: 0.0217 - mse: 0.0217 - val_loss: 0.1165 - val_mse: 0.1165
Epoch 67/150
1051/1051 [=======================

1051/1051 [==============================] - 0s 143us/step - loss: 0.0110 - mse: 0.0110 - val_loss: 0.1131 - val_mse: 0.1131
Epoch 120/150
1051/1051 [==============================] - 0s 138us/step - loss: 0.0108 - mse: 0.0108 - val_loss: 0.1160 - val_mse: 0.1160
Epoch 121/150
1051/1051 [==============================] - 0s 136us/step - loss: 0.0109 - mse: 0.0109 - val_loss: 0.1122 - val_mse: 0.1122
Epoch 122/150
1051/1051 [==============================] - 0s 138us/step - loss: 0.0109 - mse: 0.0109 - val_loss: 0.1130 - val_mse: 0.1130
Epoch 123/150
1051/1051 [==============================] - 0s 140us/step - loss: 0.0108 - mse: 0.0108 - val_loss: 0.1124 - val_mse: 0.1124
Epoch 124/150
1051/1051 [==============================] - 0s 133us/step - loss: 0.0108 - mse: 0.0108 - val_loss: 0.1130 - val_mse: 0.1130
Epoch 125/150
1051/1051 [==============================] - 0s 133us/step - loss: 0.0105 - mse: 0.0105 - val_loss: 0.1123 - val_mse: 0.1123
Epoch 126/150
1051/1051 [================

Evaluate the model (`lecun_model`) on training data (`X_train` and `y_train_scaled`) 

In [24]:
# Evaluate the model on training data
lecun_model.evaluate(X_train, y_train_scaled)

1051/1051 [==============================] - 0s 157us/step


[0.0074600337031558795, 0.0074600339867174625]

Evaluate the model (`lecun_model`) on validate data (`X_train` and `y_train_scaled`) 

In [25]:
# Evaluate the model on validate data
lecun_model.evaluate(X_val, y_val_scaled)

263/263 [==============================] - 0s 133us/step


[0.11377396267403214, 0.11377396434545517]

Not much of a difference, but a useful note to consider when tuning your network. Next, let's investigate the impact of various optimization algorithms.

## RMSprop 

Compile the `rmsprop_model` with: 

- `'rmsprop'` as the optimizer 
- track `'mse'` as the loss and metric  

In [29]:
np.random.seed(123)
rmsprop_model = Sequential()
rmsprop_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
rmsprop_model.add(layers.Dense(50, activation='relu'))
rmsprop_model.add(layers.Dense(1, activation='linear'))

# Compile the model
rmsprop_model.compile(optimizer='rmsprop', loss = 'mse', metrics = ['mse'])

# Train the model
rmsprop_model.fit(X_train, 
                  y_train_scaled, 
                  batch_size=32, 
                  epochs=150, 
                  validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 1s 530us/step - loss: 0.3446 - mse: 0.3446 - val_loss: 0.1535 - val_mse: 0.1535
Epoch 2/150
1051/1051 [==============================] - 0s 128us/step - loss: 0.1824 - mse: 0.1824 - val_loss: 0.1314 - val_mse: 0.1314
Epoch 3/150
1051/1051 [==============================] - 0s 131us/step - loss: 0.1280 - mse: 0.1280 - val_loss: 0.2049 - val_mse: 0.2049
Epoch 4/150
1051/1051 [==============================] - 0s 140us/step - loss: 0.1003 - mse: 0.1003 - val_loss: 0.1197 - val_mse: 0.1197
Epoch 5/150
1051/1051 [==============================] - 0s 126us/step - loss: 0.0876 - mse: 0.0876 - val_loss: 0.1298 - val_mse: 0.1298
Epoch 6/150
1051/1051 [==============================] - 0s 131us/step - loss: 0.0654 - mse: 0.0654 - val_loss: 0.1570 - val_mse: 0.1570
Epoch 7/150
1051/1051 [==============================] - 0s 128us/step - loss: 0.0572 - mse: 0.0572 - val_loss: 0.1314 - val_mse: 0

1051/1051 [==============================] - 0s 119us/step - loss: 0.0097 - mse: 0.0097 - val_loss: 0.1073 - val_mse: 0.1073
Epoch 61/150
1051/1051 [==============================] - 0s 121us/step - loss: 0.0093 - mse: 0.0093 - val_loss: 0.1073 - val_mse: 0.1073
Epoch 62/150
1051/1051 [==============================] - 0s 121us/step - loss: 0.0080 - mse: 0.0080 - val_loss: 0.1117 - val_mse: 0.1117
Epoch 63/150
1051/1051 [==============================] - 0s 121us/step - loss: 0.0076 - mse: 0.0076 - val_loss: 0.1059 - val_mse: 0.1059
Epoch 64/150
1051/1051 [==============================] - 0s 124us/step - loss: 0.0100 - mse: 0.0100 - val_loss: 0.1116 - val_mse: 0.1116
Epoch 65/150
1051/1051 [==============================] - 0s 121us/step - loss: 0.0053 - mse: 0.0053 - val_loss: 0.1161 - val_mse: 0.1161
Epoch 66/150
1051/1051 [==============================] - 0s 121us/step - loss: 0.0072 - mse: 0.0072 - val_loss: 0.1466 - val_mse: 0.1466
Epoch 67/150
1051/1051 [=======================

1051/1051 [==============================] - 0s 190us/step - loss: 0.0038 - mse: 0.0038 - val_loss: 0.1081 - val_mse: 0.1081
Epoch 120/150
1051/1051 [==============================] - 0s 186us/step - loss: 0.0057 - mse: 0.0057 - val_loss: 0.1162 - val_mse: 0.1162
Epoch 121/150
1051/1051 [==============================] - 0s 162us/step - loss: 0.0059 - mse: 0.0059 - val_loss: 0.1125 - val_mse: 0.1125
Epoch 122/150
1051/1051 [==============================] - 0s 162us/step - loss: 0.0049 - mse: 0.0049 - val_loss: 0.1435 - val_mse: 0.1435
Epoch 123/150
1051/1051 [==============================] - 0s 157us/step - loss: 0.0068 - mse: 0.0068 - val_loss: 0.1113 - val_mse: 0.1113
Epoch 124/150
1051/1051 [==============================] - 0s 164us/step - loss: 0.0038 - mse: 0.0038 - val_loss: 0.1120 - val_mse: 0.1120
Epoch 125/150
1051/1051 [==============================] - 0s 164us/step - loss: 0.0043 - mse: 0.0043 - val_loss: 0.1145 - val_mse: 0.1145
Epoch 126/150
1051/1051 [================

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [30]:
# Evaluate the model on training data
rmsprop_model.evaluate(X_train, y_train_scaled)

1051/1051 [==============================] - 0s 117us/step


[0.007317203365985861, 0.0073172044940292835]

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [31]:
# Evaluate the model on validate data
rmsprop_model.evaluate(X_val, y_val_scaled)

263/263 [==============================] - 0s 114us/step


[0.12013213404457831, 0.12013213336467743]

## Adam 

Compile the `adam_model` with: 

- `'Adam'` as the optimizer 
- track `'mse'` as the loss and metric  

In [32]:
np.random.seed(123)
adam_model = Sequential()
adam_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
adam_model.add(layers.Dense(50, activation='relu'))
adam_model.add(layers.Dense(1, activation='linear'))

# Compile the model
adam_model.compile(optimizer = 'Adam', loss = 'mse', metrics = ['mse'])

# Train the model
adam_model.fit(X_train, 
               y_train_scaled, 
               batch_size=32, 
               epochs=150, 
               validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 1s 514us/step - loss: 0.4873 - mse: 0.4873 - val_loss: 0.1580 - val_mse: 0.1580
Epoch 2/150
1051/1051 [==============================] - 0s 133us/step - loss: 0.1805 - mse: 0.1805 - val_loss: 0.1079 - val_mse: 0.1079
Epoch 3/150
1051/1051 [==============================] - 0s 128us/step - loss: 0.1193 - mse: 0.1193 - val_loss: 0.1065 - val_mse: 0.1065
Epoch 4/150
1051/1051 [==============================] - 0s 112us/step - loss: 0.0893 - mse: 0.0893 - val_loss: 0.1051 - val_mse: 0.1051
Epoch 5/150
1051/1051 [==============================] - 0s 119us/step - loss: 0.0761 - mse: 0.0761 - val_loss: 0.1089 - val_mse: 0.1089
Epoch 6/150
1051/1051 [==============================] - 0s 117us/step - loss: 0.0598 - mse: 0.0598 - val_loss: 0.1034 - val_mse: 0.1034
Epoch 7/150
1051/1051 [==============================] - 0s 119us/step - loss: 0.0535 - mse: 0.0535 - val_loss: 0.1087 - val_mse: 0

1051/1051 [==============================] - 0s 112us/step - loss: 5.7729e-04 - mse: 5.7729e-04 - val_loss: 0.1171 - val_mse: 0.1171
Epoch 61/150
1051/1051 [==============================] - 0s 105us/step - loss: 4.3571e-04 - mse: 4.3571e-04 - val_loss: 0.1165 - val_mse: 0.1165
Epoch 62/150
1051/1051 [==============================] - 0s 105us/step - loss: 3.9582e-04 - mse: 3.9582e-04 - val_loss: 0.1172 - val_mse: 0.1172
Epoch 63/150
1051/1051 [==============================] - 0s 102us/step - loss: 3.4343e-04 - mse: 3.4343e-04 - val_loss: 0.1164 - val_mse: 0.1164
Epoch 64/150
1051/1051 [==============================] - 0s 224us/step - loss: 4.2313e-04 - mse: 4.2313e-04 - val_loss: 0.1179 - val_mse: 0.1179
Epoch 65/150
1051/1051 [==============================] - 0s 271us/step - loss: 4.0475e-04 - mse: 4.0475e-04 - val_loss: 0.1175 - val_mse: 0.1175
Epoch 66/150
1051/1051 [==============================] - 0s 195us/step - loss: 3.1067e-04 - mse: 3.1067e-04 - val_loss: 0.1180 - val_mse

Epoch 118/150
1051/1051 [==============================] - 0s 152us/step - loss: 2.2969e-04 - mse: 2.2969e-04 - val_loss: 0.1094 - val_mse: 0.1094
Epoch 119/150
1051/1051 [==============================] - 0s 147us/step - loss: 2.2378e-04 - mse: 2.2378e-04 - val_loss: 0.1108 - val_mse: 0.1108
Epoch 120/150
1051/1051 [==============================] - 0s 150us/step - loss: 2.4077e-04 - mse: 2.4077e-04 - val_loss: 0.1084 - val_mse: 0.1084
Epoch 121/150
1051/1051 [==============================] - 0s 150us/step - loss: 3.3874e-04 - mse: 3.3874e-04 - val_loss: 0.1121 - val_mse: 0.1121
Epoch 122/150
1051/1051 [==============================] - 0s 147us/step - loss: 4.4643e-04 - mse: 4.4643e-04 - val_loss: 0.1091 - val_mse: 0.1091
Epoch 123/150
1051/1051 [==============================] - 0s 155us/step - loss: 6.1724e-04 - mse: 6.1724e-04 - val_loss: 0.1114 - val_mse: 0.1114
Epoch 124/150
1051/1051 [==============================] - 0s 147us/step - loss: 8.7244e-04 - mse: 8.7244e-04 - val_lo

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [33]:
# Evaluate the model on training data
adam_model.evaluate(X_train, y_train_scaled)

1051/1051 [==============================] - 0s 136us/step


[0.007811514070996577, 0.007811513729393482]

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [34]:
# Evaluate the model on validate data
adam_model.evaluate(X_val, y_val_scaled)

263/263 [==============================] - 0s 105us/step


[0.10366598406218758, 0.10366598516702652]

## Select a Final Model

Now, select the model with the best performance based on the training and validation sets. Evaluate this top model using the test set!

In [35]:
# Evaluate the best model on test data
rmsprop_model.evaluate(X_test, y_test_scaled)

146/146 [==============================] - 0s 411us/step


[0.10690231968278754, 0.10690231621265411]

As earlier, this metric is hard to interpret because the output is scaled. 

- Generate predictions on test data (`X_test`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_test` and `y_test_pred` 

In [37]:
# Generate predictions on test data
y_test_pred_scaled = rmsprop_model.predict(X_test)

# Transform the predictions back to original scale
y_test_pred = ss_y.inverse_transform(y_test_pred_scaled)

# MSE of test data
np.sqrt(mean_squared_error(y_test, y_test_pred))

31252.64844895025

## Summary  

In this lab, you worked to ensure your model converged properly by normalizing both the input and output. Additionally, you also investigated the impact of varying initialization and optimization routines.